In [2]:
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, OpticalElement,
                             ToroidalMirror, PlaneFilm)
from pyoptix.optimize import focus
hc = h*c/eV
pyoptix.output_notebook()

intialzing SR library
OptiX library initialized


Loading BokehJS ...

In [3]:
lambda_laser = 800e-9 # m
def harmonique(N=21):
    return lambda_laser/N

# Definition des optiques 

In [4]:
helimag = Beamline(name="Helimag")
source = GaussianSource(name="Generation")
m1T = ToroidalMirror(name="M1")
m2T = ToroidalMirror(name="M2")
foc = PlaneFilm(name="Focalisation")
focM1 = PlaneFilm(name="FocalisationM1")
spectro = PlaneFilm(name="Spectrometre")

In [5]:
helimag.chains["source"] = [source, focM1]
helimag.chains["M1"] = [source, m1T, focM1]
helimag.chains["Complete"] = [source, m1T, m2T, foc, spectro]

# Definition des paramètres optiques

## Source 

In [6]:
source.nrays = 5000
source.sigma_x = 100.e-6 # 100µm
source.sigma_y = 100.e-6 # 100µm
source.sigma_x_div = 1000.e-6 # 1 mrad
source.sigma_y_div = 1000.e-6 # 1 mrad
source.next = m1T

In [7]:
# Calcul approximatif des rayons de courbures, on prend ftot = fm1/2
dist_miroirs = 40e-3 #m
p = 1.8
q = 60e-3
ftot = 1/(1/(p + dist_miroirs/2) +1/(q + dist_miroirs/2))
f_roots = np.roots([1, -2*ftot, dist_miroirs*ftot])
print(f_roots)
f_m = f_roots[f_roots>0][0]
theta = 20*degree #rad
r = 2*f_m*np.sin(theta)
R = 2*f_m/(np.sin(theta))
print(p, q, ftot, f_m, r, R)
r= 9.32335e-2
R= 8.0318e-1
theta = 20*degree
print(f"Miroir stygmatique : {(abs(theta - np.arcsin(np.sqrt(r/R))))<1e-2}")

[0.12961399 0.02364917]
1.8 0.06 0.07663157894736843 0.12961398861389853 0.08866118992547434 0.7579319004640097
Miroir stygmatique : True


In [8]:
m1T.distance_from_previous = p # m
m1T.theta = theta # rad
m1T.phi = 90*degree # rad
m1T.minor_curvature = 1/r # m-1
m1T.major_curvature = 1/R  # m-1
m1T.next = m2T

In [9]:
m2T.distance_from_previous = dist_miroirs # m
m2T.theta = theta # rad
m2T.phi = 0*degree # rad
m2T.minor_curvature = 1/r # m-1
m2T.major_curvature = 1/R # m-1
m2T.next = foc

In [10]:
foc.distance_from_previous = q
foc.phi = -90*degree #rad
foc.next = spectro
foc.set_recording("input")

In [11]:
focM1.distance_from_previous = 0.1284
focM1.phi = -90*degree #rad
focM1.set_recording("input")

In [12]:
helimag.active_chain = "Complete"
current_foc = foc

Chaîne Complete:
	Generation -> M1 -> M2 -> Focalisation -> Spectrometre 


In [13]:
print(m1T.get_whole_parameter("minor_curvature"), 
      m1T.get_whole_parameter("major_curvature"), 
      m1T.get_whole_parameter("theta"),
      m1T.get_whole_parameter("distance")
     )
print(m1T)

spectro.distance_from_previous = 50e-3
spectro.set_recording("input")

{'value': 10.725758445194057, 'bounds': (0.0, 0.0), 'multiplier': 1.0, 'type': -1, 'group': 1, 'flags': 0} {'value': 1.2450509225827335, 'bounds': (0.0, 0.0), 'multiplier': 1.0, 'type': -1, 'group': 1, 'flags': 0} {'value': 0.3490658503988659, 'bounds': (0.0, 0.0), 'multiplier': 1.0, 'type': 1, 'group': 0, 'flags': 0} {'value': 1.8, 'bounds': (0.0, 0.0), 'multiplier': 1.0, 'type': 2, 'group': 0, 'flags': 0}
Element M1 of class <class 'pyoptix.classes.ToroidalMirror'>
	 at 1.8 m from Generation
	 pointing to M2
	 oriented in pitch at 20.0 deg (deviation 140.0 deg)
	 oriented in roll at 90.0 deg
	 oriented in yaw at 0.0 deg



helimag.active_chain="Complete"
current_foc = foc

In [14]:
helimag.clear_impacts(clear_source=True)
helimag.align(harmonique(21))
helimag.generate(harmonique(21))
helimag.radiate()

1

In [15]:
print(current_foc.beamline)

In [16]:
current_foc.show_diagram(display="all", orthonorm=False, distance_from_oe=0)

Helimag Complete


({'xy': ColumnDataSource(id='1003', ...),
  'xxp': ColumnDataSource(id='1004', ...),
  'yyp': ColumnDataSource(id='1005', ...)},
 [Column(id='1163', ...), Column(id='1321', ...), Column(id='1479', ...)])

from pyoptix.optimize import slider_optimizer
slider_optimizer(current_foc, "distance_from_previous", (0.001, 0.1), .001, screen=current_foc,  
                 wavelength=harmonique(21), display="xxp", light_spd=True, nrays=1000)

In [17]:
from pyoptix.optimize import slider_optimizer
slider_optimizer(current_foc, "distance_from_previous", (0.01, 1), .01, screen=current_foc,
                 wavelength=harmonique(21), display="xxp", light_spd=True, nrays=1000)

Helimag Complete


In [18]:
current_foc.get_diagram()

,X,Y,dX,dY,Lambda
0,4.445492e-05,-2.321237e-05,-0.031467,0.026512,3.809524e-08
1,-4.042274e-06,-1.957841e-06,0.006658,-0.004838,3.809524e-08
2,3.462153e-05,-3.037720e-06,-0.021716,-0.000049,3.809524e-08
3,3.423999e-05,-1.255369e-06,-0.023834,0.005216,3.809524e-08
4,2.022712e-05,3.162556e-07,-0.016280,-0.014216,3.809524e-08
...,...,...,...,...,...
995,-4.149470e-07,-4.510306e-06,-0.000603,-0.019653,3.809524e-08
996,3.513419e-05,-1.522245e-05,-0.025008,0.023720,3.809524e-08
997,3.935462e-06,-1.051332e-05,0.017526,-0.010088,3.809524e-08
998,-3.180901e-07,2.459235e-06,0.003622,-0.007258,3.809524e-08


In [19]:
print(current_foc)

Element Focalisation of class <class 'pyoptix.classes.PlaneFilm'>
	 at 0.06 m from M2
	 pointing to Spectrometre
	 oriented in pitch at 0.0 deg (deviation 180.0 deg)
	 oriented in roll at -90.0 deg
	 oriented in yaw at 0.0 deg



In [20]:
1.8*1/np.sin(20*degree)

5.262847920293558